#### Aloca fluxo de caixa entre Tranche Sênior e Subordinada.  
  
####    Parâmetros:  
#####    -----------  
#####    cashflow_real : pd.DataFrame  
#####    DataFrame com colunas ['month', 'fluxo_real']  
#####    senior_perc : float  
#####    Percentual do fluxo destinado à Tranche Sênior  
  
####    Retorno:  
#####    --------  
#####    pd.DataFrame com colunas:  
#####    ['month', 'cash_in', 'senior_paid', 'subordinated_paid',   
#####     'senior_outstanding', 'sub_outstanding']

In [8]:
import os
import pandas as pd

# Importa simulaçao e metricas
import import_ipynb
from Simulate_Portfolio import simulate_portfolio
from Calculate_Metrics import calculate_metrics

✅ CSVs encontrados. Carregando...
Saldo Total: 380375223.97338593
Taxa Inadimplência: 0.1787
Retorno Esperado (IRR mensal): nan

Fluxo Caixa Agregado:
    month  fluxo_aggregado
0      1     1.280811e+07
1      2     1.275159e+07
2      3     1.270145e+07
3      4     1.264862e+07
4      5     1.257817e+07

Fluxo Caixa Real:
    month    fluxo_real
0      1  1.372814e+07
1      2  1.363886e+07
2      3  1.351314e+07
3      4  1.331599e+07
4      5  1.361498e+07


In [9]:
def allocate_waterfall(cashflow_real: pd.DataFrame, senior_perc: float = 0.8):

    waterfall_rows = []
    senior_outstanding = 0.0
    sub_outstanding = 0.0

    for _, row in cashflow_real.iterrows():
        month = int(row["month"])
        cash_in = float(row["fluxo_real"])

        # Alocação do fluxo
        senior_due = cash_in * senior_perc
        senior_paid = min(cash_in, senior_due)
        subordinated_paid = max(0.0, cash_in - senior_paid)

        # Atualiza acumulados
        senior_outstanding += senior_paid
        sub_outstanding += subordinated_paid

        waterfall_rows.append({
            "month": month,
            "cash_in": cash_in,
            "senior_paid": senior_paid,
            "subordinated_paid": subordinated_paid,
            "senior_outstandig": senior_outstanding,
            "sub_outstanding": sub_outstanding
        })
    
    return pd.DataFrame(waterfall_rows)

In [10]:
# Garante diretório  
os.makedirs("data", exist_ok=True)  
  
# Caminhos  
portfolio_path = "data/portifolio.csv"  
cashflows_path = "data/cashflows.csv"  
  
# Se não houver dados, roda simulação  
if not (os.path.exists(portfolio_path) and os.path.exists(cashflows_path)):  
    print("⚠️  CSVs não encontrados. Gerando simulação...")  
    df_portfolio, df_cashflows = simulate_portfolio(n_contracts=10_000, seed=42)  
    df_portfolio.to_csv(portfolio_path, index=False)  
    df_cashflows.to_csv(cashflows_path, index=False)  
else:  
    print("✅ CSVs encontrados. Carregando...")  
    df_portfolio = pd.read_csv(portfolio_path)  
    df_cashflows = pd.read_csv(cashflows_path)

✅ CSVs encontrados. Carregando...


In [ ]:
# Calcula Metricas
# metrics = calculate_metrics(df_portfolio, df_cashflows)
# cashflow_real = metrics["fluxo_caixa_real"]

# Executa Waterfall
# waterfall_df = allocate_waterfall(cashflow_real, senior_perc=0.8)

In [ ]:
# print(">> Saída do Waterfall:")  
# print(waterfall_df.head())

>> Saída do Waterfall:
   month       cash_in   senior_paid  subordinated_paid  senior_outstandig  \
0      1  1.372814e+07  1.098251e+07       2.745627e+06       1.098251e+07   
1      2  1.363886e+07  1.091109e+07       2.727772e+06       2.189360e+07   
2      3  1.351314e+07  1.081051e+07       2.702627e+06       3.270411e+07   
3      4  1.331599e+07  1.065279e+07       2.663199e+06       4.335690e+07   
4      5  1.361498e+07  1.089199e+07       2.722996e+06       5.424889e+07   

   sub_outstanding  
0     2.745627e+06  
1     5.473399e+06  
2     8.176027e+06  
3     1.083923e+07  
4     1.356222e+07  
